In [1]:
%matplotlib inline
import glob, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 200

plt.rcParams['figure.figsize'] = [20, 10]
plt.rc('font', size=20)



# Loading Dataset

## IOT Devices

The dataset has been extracted at `ROOT_PATH`, containing the data for each IOT device. There are 9 devices in total. There are folders for each IOT device containing `benign_traffic.csv` and further two folders for `gafgyt_attacks` and `mirai_attacks`

In [ ]:
ROOT_PATH = r'/mnt/data/khiz/dataset/00442'

IOT_DEVS = [ 'Danmini_Doorbell',
             'Ecobee_Thermostat',
#              'Ennio_Doorbell',
#              'B120N10_Baby_Mon',
#              '737E_Security_Cam',
#              '838_Security_Cam',
#              'Samsung_Webcam',
#              '1002_Security_Cam',
#              '1003_Security_Cam'
           ]

## Paths to CSVs
We create a python dictionary that contains all the paths to the CSVs for the respective IOT devices

In [ ]:
PATHS_DICT = {}

for i in IOT_DEVS:
    PATHS_DICT[i] = {}
    print('[' + i + ']')
    iot_dir = os.path.join(ROOT_PATH, i)
    PATHS_DICT[i]['benign'] = os.path.join(iot_dir, 'benign_traffic.csv')
    print('  ', 'benign_traffic.csv')
    attacks = [ d for d in os.listdir(iot_dir)
                   if os.path.isdir( os.path.join(iot_dir, d)) ]
    for attack in attacks:
        attack_name = attack.split('_')[0]
        
        PATHS_DICT[i][attack_name] = {}
        attack_dir = os.path.join(iot_dir, attack)
        types = [ f for f in os.listdir(attack_dir)
                   if os.path.isfile( os.path.join(attack_dir, f)) ]
        print('  ', attack_name, ': ', types)
        for t in types:
            type_name = t.split('.')[0]
            PATHS_DICT[i][attack_name][type_name] = os.path.join(attack_dir,t)

## Loading CSVs into Pandas
We will create pandas dataframe for each botnet attack on each IOT device. For each datafram we will combine the attack traffic with the benign traffic add the following two columns:
- `traffic_type` : benign or attack (0 or 1)
- `attack_type`  : type of attack e.g, ack, scan etc.

In [ ]:
nbaiot_dict = {}
for iot_dev in PATHS_DICT:
    b_df = pd.read_csv(PATHS_DICT[iot_dev]['benign'])
    b_df['traffic_type'] = 'benign'
    b_df['attack_type'] = 'benign'
    for botnet in [ b for b in PATHS_DICT[iot_dev] if b != 'benign']:
        if botnet not in nbaiot_dict:
            nbaiot_dict[botnet] = pd.DataFrame()
        nbaiot_dict[botnet] = nbaiot_dict[botnet].append(b_df)
        for attack in PATHS_DICT[iot_dev][botnet]:
            a_df = pd.read_csv(PATHS_DICT[iot_dev][botnet][attack])
            a_df['traffic_type'] = 'attack'
            a_df['attack_type'] = attack
            nbaiot_dict[botnet] = nbaiot_dict[botnet].append(a_df)

In [ ]:
for k in nbaiot_dict:
    print(k,':','<benign+attack>')

In [ ]:
nbaiot_dict['gafgyt'].shape

In [ ]:
nbaiot_dict['mirai'].shape

In [ ]:
nbaiot_dict['gafgyt']['traffic_type'].value_counts()

In [ ]:
nbaiot_dict['mirai']['traffic_type'].value_counts()

# Modeling

### Pre processing
Convert the target column `traffic_type` to integer 0 = benigh, 1 = attack 

In [ ]:
for botnet in nbaiot_dict:
    nbaiot = nbaiot_dict[botnet]

    nbaiot.loc[ nbaiot['traffic_type']=='attack','traffic_type' ]=1
    nbaiot.loc[ nbaiot['traffic_type']=='benign','traffic_type' ]=0
    nbaiot['traffic_type'] = nbaiot['traffic_type'].astype(int)

    nbaiot_dict[botnet] = nbaiot

### Classifiers and training functions

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [ ]:
classfiers = {
    'KNN' : KNeighborsClassifier(),
    'RFR' : RandomForestClassifier(),
    'DTR' : DecisionTreeClassifier(),
    'ADB' : AdaBoostClassifier(),
    
}

def train_test_report( X, y, test_size=0.33, avg='macro avg' ):

    X_train, X_test, y_train, y_test = train_test_split(
                                        X, y, test_size=0.33, random_state=42)

    result = []
    for clf_name in classfiers:
        clf = classfiers[clf_name]
        clf.fit(X_train, y_train)
        
        y_preds = clf.predict(X_test)
        
        clrp = classification_report(y_test, y_preds, output_dict=True)
        conf_mat = confusion_matrix(y_test, y_preds)
        fp = conf_mat[0][1]
        fn = conf_mat[1][0]
        
        result.append([
            clf_name,
            clrp['accuracy'],
            clrp[avg]['precision'],
            clrp[avg]['recall'],
            clrp[avg]['f1-score'],
            fp,
            fn
        ])

    return result

## Main Loop

In [ ]:
all_results = [
    ['BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N' ]
]

for botnet in nbaiot_dict:
    nbaiot = nbaiot_dict[botnet]

    print("================", botnet, "================")

    X = nbaiot.drop( ["attack_type", "traffic_type" ], axis=1 )
    y = nbaiot["traffic_type"]

    rep = train_test_report( X, y)
    for r in rep:
        print(r)
        all_results.append( [botnet] + r )

    print("================================================================")
    print("")


In [ ]:
res_df =pd.DataFrame(all_results, index=None)

res_df, res_df.columns = res_df[1:] , res_df.iloc[0]

res_df

In [ ]:
# Write to csv
res_csv = res_df.to_csv(index=False)

print(res_csv, file=open('03_results.csv', 'w'))


### Load results from CSV

In [ ]:
res_df = pd.read_csv('03_results.csv')

res_df = res_df[['BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'FALSE-P', 'FALSE-N' ]]

res_df

In [ ]:
res_df.set_index(['BOTNET', 'CLASSIFIER'], inplace=True)
res_df

In [ ]:
res_df.plot(kind='bar')